In [1]:
import pandas as pd

# Read with semicolon delimiter
df = pd.read_csv('../../data/LD2011_2014.txt', delimiter=';', index_col=0, dtype=str)
#df = pd.read_csv('../../data/household_power_consumption.txt', delimiter=';', index_col=0, dtype=str)
df = df.apply(lambda c: pd.to_numeric(c.str.replace(",", ".", regex=False),
                                      errors="coerce"))
#df = df.iloc[-10000:]

# Write as standard CSV (comma-delimited)
#df.to_csv('LD2011_2014.csv', index=False)

In [2]:
df.shape

(140256, 370)

In [3]:
name_feat = "MT_001"
#name_feat = "Global_active_power"
s = df[name_feat]

mask = s.ne(0) & s.notna()
first_idx = s.index[mask.argmax()] if mask.any() else None
first_idx

'2012-01-01 00:15:00'

In [4]:
df["MT_001"].loc["2014-06-30 09:00:00":"2014-07-01 00:00:00"]

2014-06-30 09:00:00     0.000000
2014-06-30 09:15:00     1.269036
2014-06-30 09:30:00     0.000000
2014-06-30 09:45:00     3.807107
2014-06-30 10:00:00    10.152284
                         ...    
2014-06-30 23:00:00     2.538071
2014-06-30 23:15:00     1.269036
2014-06-30 23:30:00     1.269036
2014-06-30 23:45:00     2.538071
2014-07-01 00:00:00     1.269036
Name: MT_001, Length: 61, dtype: float64

In [5]:
import numpy as np

#first_idx = "2014-06-30 09:00:00"

df_from = df.loc[first_idx:] if first_idx is not None else df.iloc[0:0]
all_x = df_from[name_feat].diff().dropna().values.reshape(-1,1)
#mask = (all_x[1:] > 0) & (all_x[:-1] > 0)
#log_ret = np.log(all_x[1:][mask] / all_x[:-1][mask])
#log_ret = log_ret.reshape(-1,1)
log_ret = all_x


In [6]:
df_from[name_feat].diff().dropna().values.reshape(-1,1)[13]

array([-1.26903553])

In [7]:

def standarize(exog):
    norm_exog = np.zeros((1,exog.shape[1]-1))
    mean_st = []
    std_st = []
    for i in range(2,exog.shape[1]):
        norm_exog[0,i-2] = (exog[0,i-1] - exog[0,:i].mean())/exog[0,:i].std()
        mean_st.append(exog[0,:i].mean())
        std_st.append(exog[0,:i].std())
    
    return norm_exog.T,mean_st,std_st


norm_exog,mean_st,std_st =  standarize(log_ret.T)
tol = 2.22e-16
norm_exog[np.abs(norm_exog) < tol] = 0.0
norm_exog = norm_exog[~np.isnan(norm_exog)].reshape(-1,1)

In [8]:
df_lags = pd.DataFrame(norm_exog)
df_lags.to_csv("../../data/electricity/raw_norm_LD2011_2014.csv", index=False)

In [8]:
norm_exog.shape

(105214, 1)

In [9]:
lag_matrix = lambda x,lag: np.array([[x[j+i, 0] for j in range(lag)] for i in range(x.shape[0]-lag)])
lags = 48
df_lag_m = lag_matrix(norm_exog, lags)
target_t_0 = norm_exog[lags:]

In [10]:
df_lag_m

array([[-1.00000000e+00,  0.00000000e+00,  9.04534034e-01, ...,
        -4.61944257e-01,  9.93709851e-03,  2.62966442e+00],
       [ 0.00000000e+00,  9.04534034e-01, -1.11803399e+00, ...,
         9.93709851e-03,  2.62966442e+00, -4.85247479e-01],
       [ 9.04534034e-01, -1.11803399e+00,  0.00000000e+00, ...,
         2.62966442e+00, -4.85247479e-01, -2.15269698e+00],
       ...,
       [ 4.23959769e-06, -4.45850511e-01,  8.47910675e-06, ...,
        -4.45928601e-01,  4.45943014e-01,  4.23849841e-06],
       [-4.45850511e-01,  8.47910675e-06,  8.47906643e-06, ...,
         4.45943014e-01,  4.23849841e-06,  4.23847827e-06],
       [ 8.47910675e-06,  8.47906643e-06,  4.45869167e-01, ...,
         4.23849841e-06,  4.23847827e-06, -4.45936235e-01]])

In [11]:
np.isnan(df_lag_m).any()

False

In [12]:
df_lags = pd.DataFrame(df_lag_m)
df_lags.to_csv("../../data/electricity/lags_LD2011_2014.csv", index=False)

df_target = pd.DataFrame(target_t_0)
df_target.to_csv("../../data/electricity/target_LD2011_2014.csv", index=False)

In [13]:
df_lags.iloc[:5]

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,-1.000000,0.000000,0.904534,-1.118034,0.000000,1.664101,-1.632993,0.742781,-0.845154,0.000000,...,0.453195,1.012016,-1.283155,-0.122702,0.457128,-0.713746,-4.212227,-0.461944,0.009937,2.629664
1,0.000000,0.904534,-1.118034,0.000000,1.664101,-1.632993,0.742781,-0.845154,0.000000,0.822179,...,1.012016,-1.283155,-0.122702,0.457128,-0.713746,-4.212227,-0.461944,0.009937,2.629664,-0.485247
2,0.904534,-1.118034,0.000000,1.664101,-1.632993,0.742781,-0.845154,0.000000,0.822179,-0.071796,...,-1.283155,-0.122702,0.457128,-0.713746,-4.212227,-0.461944,0.009937,2.629664,-0.485247,-2.152697
3,-1.118034,0.000000,1.664101,-1.632993,0.742781,-0.845154,0.000000,0.822179,-0.071796,-0.935414,...,-0.122702,0.457128,-0.713746,-4.212227,-0.461944,0.009937,2.629664,-0.485247,-2.152697,-0.418942
4,0.000000,1.664101,-1.632993,0.742781,-0.845154,0.000000,0.822179,-0.071796,-0.935414,0.878438,...,0.457128,-0.713746,-4.212227,-0.461944,0.009937,2.629664,-0.485247,-2.152697,-0.418942,0.447306


In [14]:
df_lags

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,-1.000000,0.000000,0.904534,-1.118034,0.000000,1.664101,-1.632993,0.742781,-0.845154,0.000000,...,0.453195,1.012016,-1.283155,-0.122702,0.457128,-0.713746,-4.212227,-0.461944,0.009937,2.629664
1,0.000000,0.904534,-1.118034,0.000000,1.664101,-1.632993,0.742781,-0.845154,0.000000,0.822179,...,1.012016,-1.283155,-0.122702,0.457128,-0.713746,-4.212227,-0.461944,0.009937,2.629664,-0.485247
2,0.904534,-1.118034,0.000000,1.664101,-1.632993,0.742781,-0.845154,0.000000,0.822179,-0.071796,...,-1.283155,-0.122702,0.457128,-0.713746,-4.212227,-0.461944,0.009937,2.629664,-0.485247,-2.152697
3,-1.118034,0.000000,1.664101,-1.632993,0.742781,-0.845154,0.000000,0.822179,-0.071796,-0.935414,...,-0.122702,0.457128,-0.713746,-4.212227,-0.461944,0.009937,2.629664,-0.485247,-2.152697,-0.418942
4,0.000000,1.664101,-1.632993,0.742781,-0.845154,0.000000,0.822179,-0.071796,-0.935414,0.878438,...,0.457128,-0.713746,-4.212227,-0.461944,0.009937,2.629664,-0.485247,-2.152697,-0.418942,0.447306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105161,0.000008,0.445859,0.000004,-0.445851,0.000008,0.000008,0.445869,-0.445858,0.000008,0.000008,...,0.000004,0.000004,-0.445916,0.445931,0.445928,-0.445926,-0.445923,0.445937,0.000004,-0.445929
105162,0.445859,0.000004,-0.445851,0.000008,0.000008,0.445869,-0.445858,0.000008,0.000008,0.445877,...,0.000004,-0.445916,0.445931,0.445928,-0.445926,-0.445923,0.445937,0.000004,-0.445929,0.445943
105163,0.000004,-0.445851,0.000008,0.000008,0.445869,-0.445858,0.000008,0.000008,0.445877,-0.445866,...,-0.445916,0.445931,0.445928,-0.445926,-0.445923,0.445937,0.000004,-0.445929,0.445943,0.000004
105164,-0.445851,0.000008,0.000008,0.445869,-0.445858,0.000008,0.000008,0.445877,-0.445866,0.000008,...,0.445931,0.445928,-0.445926,-0.445923,0.445937,0.000004,-0.445929,0.445943,0.000004,0.000004
